<h1 class="title">Jason Scott's favorite 100 - data analysis using NLP</h1>
<h2 class="subtitle">Mini-Project in the ABC of Computational Text Analysis</h2>
<p>Author: <span class="author">Josias Bruderer, Universität Luzern</span></p>
<p>Date: <span class="date">14. May 2021</span></p>

# Preparations

The following lines of code is used for preparing our environment.

In [1]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

# load the necessary libraries

import sys
from pathlib import Path

# prepare to load project specific libraries
module_path = str(Path.cwd() / "py")
if module_path not in sys.path:
    sys.path.append(module_path)

# import data_wrangler module
from modules import data_wrangler

# Data Wrangling

In this section the required data is downloaded and preprocessed (f.E. unzipped). The module data_wrangler will be used for this.

In [2]:
data_url = "http://archives.textfiles.com/100.zip"
data_dir = sys.path[0] + "/data/"
tmp_dir = sys.path[0] + "/.tmp/"

dw = data_wrangler.DataWrangler(tmp_dir, data_dir)

dw.process_zip(data_url, "100")